In [2]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
proj_dir = os.path.abspath('../..')

if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))

import numpy as np
import scipy.stats as stats
import pandas as pd
from scipy.spatial import distance
from random import random
from sklearn.cluster import SpectralBiclustering

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
from matplotlib import colors

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import blockworld_helpers as utils
import drawing_utils as drawing
import importlib
import scoring
import rda

In [3]:
## directory & file hierarchy
proj_dir = os.path.abspath('../..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.path.abspath(os.path.join(os.getcwd(),'..'))
results_dir = os.path.join(analysis_dir,'results')
stim_dir = os.path.join(proj_dir,'stimuli')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))
jefan_dir = os.path.join(analysis_dir,'jefan')
will_dir = os.path.join(analysis_dir,'will')

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [4]:
stim_dir

'/home/wmccarth/block_construction/stimuli'

In [5]:
iteration_name = 'Exp2Pilot3'
num_trials = 24 #for sanity checks

In [6]:
# Data already compiled into dataframes in CogSci 2020 Dataframe Generator

# trial_end data
trial_path = os.path.join(csv_dir,'block_silhouette_{}_good.csv'.format(iteration_name))
df = pd.read_csv(trial_path)

# # initial_block data
initial_path = os.path.join(csv_dir,'block_silhouette_initial_{}_good.csv'.format(iteration_name))
dfi = pd.read_csv(initial_path)

# # settled_block data
settled_path = os.path.join(csv_dir,'block_silhouette_settled_{}_good.csv'.format(iteration_name))
dfs = pd.read_csv(settled_path)

# # Sanity Check- same participants in each dataset.
df_participants = df.gameID.unique()
dfs_participants = dfs.gameID.unique()
assert Counter(df_participants) == Counter(dfs_participants)

n_before_outliers = len(df_participants)
print(str(n_before_outliers) + ' participants total')

dfi['usableDiscreteWorld'] = dfi['discreteWorld'].apply(lambda a: 1+(-1)*np.array(ast.literal_eval(a)))
dfi['flatDiscreteWorld'] = dfi['discreteWorld'].apply(lambda a: (1+(-1)*np.array(ast.literal_eval(a))).flatten())

49 participants total


In [7]:
dfi.columns

Index(['Unnamed: 0', 'gameID', 'trialNum', 'phase', 'condition', 'eventType',
       'targetName', 'repetition', 'targetID', 'blockNum', 'nullScore',
       'incrementalScore', 'normedIncrementalScore', 'rawScoreDiscrete',
       'incrementalNormedScoreDiscretePrevious', 'score', 'currBonus',
       'timeAbsolute', 'timeRelative', 'timeBlockSelected', 'timeBlockPlaced',
       'relativePlacementTime', 'discreteWorld', 'vertices', 'blockKind',
       'blockColorID', 'blockColor', 'blockCenterX', 'blockCenterY', 'x_index',
       'y_index', 'x_discrete', 'y_discrete', 'width_discrete',
       'height_discrete', 'buggy', 'usableDiscreteWorld', 'flatDiscreteWorld'],
      dtype='object')

## Actions

Placement of block: <w,h,x,y>

Only certain placements are possible at any one time: i.e. those just above the current tower (including in holes).

Consider set of actions as possible block placements *inside the silhouette* plus an error action- placing a block anywhere outside the silhouette.


In [8]:
blocks = np.array([[1,2],[2,1],[2,2],[2,4],[4,2]]) # block dimensions
locations = np.arange(0,8) # locations to place blocks- inside 8x8 grid only

### Maps of silhouette

In [9]:
target_dir = os.path.join(stim_dir,'hand_selected_e2_subset')

## load in list of structures
file_list = os.listdir(target_dir)
file_list = [x for x in file_list if not (x.startswith('.'))]

print('Loaded {} structures.'.format(len(file_list)))

target_blocks_dict = {}

## loop through list of good sequences, and build list of versions, each containing a trial list 
Meta = [] ## initialize list of all trial lists
for i,f in enumerate(file_list):
    target = pd.read_json(os.path.join(target_dir,f)) ## stim list
    targetName = f.split('.')[0]
    targetBlocks = list(target['blocks'].values)
    target_blocks_dict[targetName] = targetBlocks

Loaded 8 structures.


In [ ]:
stimWidth = 8
worldWidth = 18
worldHeight = 13

target_maps = {}

for i, (target_name, block_list) in enumerate(target_blocks_dict.items()):
    #block_list = target_blocks_dict['hand_selected_009']
    target_map = np.zeros([worldWidth,worldHeight])

    for block in block_list:
        # add block to map
        width = block['width']
        height = block['height']
        blockLeft = int((worldWidth-stimWidth)/2) + block['x']
        blockBottom = block['y']
        blockTop = blockBottom + height
        blockRight = blockLeft + width

        #console.log('width', width);
        #console.log('height', height);
        #console.log('blockLeft', blockLeft);
        #console.log('blockBottom', blockBottom);
        for y in range(blockBottom,blockTop):
            for x in range(blockLeft, blockRight):
                target_map[x][y] = 1
    
    target_maps[target_name] = target_map
    

In [ ]:
for i, (target_name, target_map) in enumerate(target_maps.items()):
    silhouette = 1*np.logical_not(target_map)
    silhouette = np.rot90(silhouette)
    fig, ax = plt.subplots()
    ax.axis('off')
    ax.imshow(silhouette)

### Enumerate actions within silhouette

In [ ]:
silhouette = 1*np.logical_not(target_maps['hand_selected_012'])
silhouette = np.rot90(silhouette)
fig, ax = plt.subplots()
ax.axis('off')
ax.imshow(silhouette)

In [ ]:
tm = target_maps['hand_selected_012']

In [ ]:
tm[5:12,0:8] # Slice only the silhouette

for a current world state:
find set of viable actions

iterate through layers (keep track of floors?)
for each block find viable locations
or 
for each location find viable blocks

some of these paths will lead to dead ends

## Actual distributions of human behavior

Look at the sequences of actions people actually take
And see how close they are
Do they end up closer (by rep 4)?

Similarity:
Closer if: 
Same blocks in same place
Same blocks in same placed placed at similar time

In [ ]:
dfi['usableDiscreteWorld'] = dfi['discreteWorld'].apply(lambda a: 1+(-1)*np.array(ast.literal_eval(a)))
dfi['flatDiscreteWorld'] = dfi['discreteWorld'].apply(lambda a: (1+(-1)*np.array(ast.literal_eval(a))).flatten())

In [ ]:
# We have sequences of world states, of length n where n is the number of blocks placed
# Each layer is a block placement
dfi['flatDiscreteWorld'][0]

Vector representing where blocks have been placed
Want to look at sequences of these
If people complete the silhouette, these vectors will be the same
Differing numbers of blocks mean the list of these vectors will be different lengths

## Get world states by time step

In [ ]:
# get world state EVERY SECOND
# i.e. the flatDiscreteWorld for the highest time preceding i 
# for every participant, for every structure

targets = np.sort(df['targetName'].unique())
ppts = np.sort(df['gameID'].unique())

n_targets = df['targetName'].nunique()
n_ppts = df['gameID'].nunique()

m = np.zeros((n_targets,n_ppts,61,13*18)) #number of seconds, number of squares in grid

for t, t_name in enumerate(targets):
    for p, p_id in enumerate(ppts):
        subset = dfi[(dfi.targetName==t_name) & (dfi.gameID==p_id) & (dfi.phase=='pre')]
        prev_world = np.zeros(13*18)
        i = 0
        for index, row in subset.iterrows():
            while ((i*1000 < row['relativePlacementTime']) & ((i*1000)<60000)):
                m[t,p,i,:] = prev_world
                i += 1
            prev_world = row['flatDiscreteWorld']
        while (i < 60):
            m[t,p,i,:] = prev_world
            i += 1


In [ ]:
# get world state EVERY 10 SECONDS
# i.e. the flatDiscreteWorld for the highest time preceding i 
# PRE PHASE, for every participant, for every structure

targets = np.sort(df['targetName'].unique())
ppts = np.sort(df['gameID'].unique())

n_targets = df['targetName'].nunique()
n_ppts = df['gameID'].nunique()

m = np.zeros((n_targets,n_ppts,6,13*18)) #number of seconds, number of squares in grid

for t, t_name in enumerate(targets):
    for p, p_id in enumerate(ppts):
        subset = dfi[(dfi.targetName==t_name) & (dfi.gameID==p_id) & (dfi.phase=='pre')]
        prev_world = np.zeros(13*18)
        i = 0
        for index, row in subset.iterrows():
            while ((i*10000 < row['relativePlacementTime']) & ((i*10000)<60000)):
                m[t,p,i,:] = prev_world
                i += 1
            prev_world = row['flatDiscreteWorld']
        while (i < 6):
            m[t,p,i,:] = prev_world
            i += 1


In [ ]:
# get world state EVERY 10 SECONDS
# i.e. the flatDiscreteWorld for the highest time preceding i 
# POST PHASE, for every participant, for every structure

targets = np.sort(df['targetName'].unique())
ppts = np.sort(df['gameID'].unique())

n_targets = df['targetName'].nunique()
n_ppts = df['gameID'].nunique()

m2 = np.zeros((n_targets,n_ppts,6,13*18)) #number of seconds, number of squares in grid

for t, t_name in enumerate(targets):
    for p, p_id in enumerate(ppts):
        subset = dfi[(dfi.targetName==t_name) & (dfi.gameID==p_id) & (dfi.phase=='post')]
        prev_world = np.zeros(13*18)
        i = 0
        for index, row in subset.iterrows():
            while ((i*10000 < row['relativePlacementTime']) & ((i*10000)<60000)):
                m2[t,p,i,:] = prev_world
                i += 1
            prev_world = row['flatDiscreteWorld']
        while (i < 6):
            m2[t,p,i,:] = prev_world
            i += 1


In [ ]:
# (targets, participants, seconds, squares)
# for one target, get all participants
mat = m[3,:,50,:]

## Explore matrix creation (compare ppt world state for a given structure over time)

In [ ]:
# Create rdm for a given structure and time step (all ppts, one phase)

mat = m[3,:,1,:]
n_rows = mat.shape[0]

rdm = np.zeros((n_rows,n_rows))

for i in range (0, n_rows): # row dim
    for j in range (0, n_rows): # row dim
        rdm[i,j] = distance.euclidean(mat[i,:],mat[j,:])
        

img = plt.matshow(rdm)
img.set_cmap('hot')
plt.axis('off')
plt.savefig("test.png", bbox_inches='tight')
plt.colorbar()

In [ ]:
# Use biclustering to arrange rows and columns. 
# This seems ok, except you have to specify the amount of clusters and the initial random state seems to have quite a large effect

mat = m[1,:,2,:]
n_rows = mat.shape[0]

rdm = np.zeros((n_rows,n_rows))

for i in range (0, n_rows): # row dim
    for j in range (0, n_rows): # row dim
        rdm[i,j] = distance.euclidean(mat[i,:],mat[j,:])

clustering = SpectralBiclustering(n_clusters=5, random_state=0).fit(rdm) # https://scikit-learn.org/stable/auto_examples/bicluster/plot_spectral_biclustering.html
order = clustering.row_labels_

sorted_rdm = rdm[np.argsort(clustering.row_labels_)]
sorted_rdm = sorted_rdm[:, np.argsort(clustering.column_labels_)]

img1 = plt.matshow(rdm)
plt.axis('off')
plt.colorbar()
img2 = plt.matshow(sorted_rdm)
plt.axis('off')
img1.set_cmap('hot')
img2.set_cmap('hot')
plt.colorbar()

In [ ]:
importlib.reload(scoring)

In [ ]:
# Create and display RDMs for one structure (all ppts, one phase)

mat = m[3,:,:,:]

n_rows = mat.shape[0]
time_steps = mat.shape[1]

rdm = np.zeros((n_rows,n_rows, time_steps))

for step in range(0,time_steps):
    for i in range (0, n_rows): # row dim
        for j in range (0, n_rows): # row dim
            rdm[i,j,step] = scoring.get_jaccard(mat[i,step,:],mat[j,step,:])
        
for step in range(0,time_steps):
    img = plt.matshow(rdm[:,:,step])
    img.set_cmap('hot')
    plt.axis('off')
    plt.savefig("test.png", bbox_inches='tight')
    plt.colorbar()

## Sequences of world states by action

In [ ]:
dfic['flatDiscreteWorld'].apply(lambda x: x[100])

In [27]:
dfic = dfi[['targetName','gameID','blockNum','condition','repetition','flatDiscreteWorld']]

inds = list([])

for i in range(0,len(dfi['flatDiscreteWorld'][0])):
    inds.append('w_' + str(i).zfill(3))
    #assign_dict['w_' + str(i).zfill(3)] = lambda x: x['flatDiscreteWorld'].apply(lambda row: row[i])


#world_cols = dfic.apply(lambda x: x['flatDiscreteWorld'], axis=1, result_type='expand')
world_cols = dfic.apply(lambda x: pd.Series(x['flatDiscreteWorld'], index=inds), axis=1)
dfic = pd.concat([dfic, world_cols], axis=1)
dfic

,targetName,gameID,blockNum,condition,repetition,flatDiscreteWorld,w_000,w_001,w_002,w_003,...,w_224,w_225,w_226,w_227,w_228,w_229,w_230,w_231,w_232,w_233
0,hand_selected_009,0145-8cab6010-6b88-4e57-b04d-b9e9b4b7c959,1,repeated,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,hand_selected_009,0145-8cab6010-6b88-4e57-b04d-b9e9b4b7c959,2,repeated,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,hand_selected_009,0145-8cab6010-6b88-4e57-b04d-b9e9b4b7c959,3,repeated,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,hand_selected_009,0145-8cab6010-6b88-4e57-b04d-b9e9b4b7c959,4,repeated,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,hand_selected_009,0145-8cab6010-6b88-4e57-b04d-b9e9b4b7c959,5,repeated,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,hand_selected_009,0145-8cab6010-6b88-4e57-b04d-b9e9b4b7c959,6,repeated,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,hand_selected_005,0145-8cab6010-6b88-4e57-b04d-b9e9b4b7c959,1,control,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,hand_selected_005,0145-8cab6010-6b88-4e57-b04d-b9e9b4b7c959,2,control,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,hand_selected_005,0145-8cab6010-6b88-4e57-b04d-b9e9b4b7c959,3,control,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,hand_selected_005,0145-8cab6010-6b88-4e57-b04d-b9e9b4b7c959,4,control,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
assign_dict

In [28]:
dfic.loc[20,'flatDiscreteWorld']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [34]:
np.arraydfic.loc[:,'w_000':'w_233'].loc[20]

w_000    0
w_001    0
w_002    0
w_003    0
w_004    0
w_005    0
w_006    0
w_007    0
w_008    0
w_009    0
w_010    0
w_011    0
w_012    0
w_013    0
w_014    0
w_015    0
w_016    0
w_017    0
w_018    0
w_019    0
w_020    0
w_021    0
w_022    0
w_023    0
w_024    0
w_025    0
w_026    0
w_027    0
w_028    0
w_029    0
        ..
w_204    0
w_205    0
w_206    0
w_207    0
w_208    0
w_209    0
w_210    0
w_211    0
w_212    0
w_213    0
w_214    0
w_215    0
w_216    0
w_217    0
w_218    0
w_219    0
w_220    0
w_221    0
w_222    0
w_223    0
w_224    0
w_225    0
w_226    0
w_227    0
w_228    0
w_229    0
w_230    0
w_231    0
w_232    0
w_233    0
Name: 20, Length: 234, dtype: int64

In [ ]:
## make tidy dataframe of world states

# Create dictionary of lambda functions that grabs ith value and puts it in column w_i
assign_dict = {}
n_squares = len(dfi['flatDiscreteWorld'][0])
for i in range(0, n_squares):
    #assign_dict['w_' + str(i).zfill(3)] = lambda x:  x['flatDiscreteWorld'].apply(lambda row: row[i])
    assign_dict['w_' + str(i).zfill(3)] = lambda x:  (x.flatDiscreteWorld.apply(lambda row: row[i], axis=1))

dfic = dfi[['targetName','gameID','blockNum','condition','repetition','flatDiscreteWorld']]
dfic = dfic.assign(**assign_dict)
dfic

In [ ]:
assign_dict

In [ ]:
## make tidy dataframe of world states

# Create dictionary of lambda functions that grabs ith value and puts it in column w_i
assign_dict = {}
n_squares = len(dfi['flatDiscreteWorld'][0])
for i in range(0, n_squares):
    assign_dict['w_' + str(i).zfill(3)] = lambda x:  (x['flatDiscreteWorld'].apply(lambda row: row[i], axis=1))
    #assign_dict['w_' + str(i).zfill(3)] = lambda x:  x.flatDiscreteWorld[i]


dfic = dfi[['targetName','gameID','blockNum','condition','repetition','flatDiscreteWorld']]
dfic = dfic.assign(**assign_dict)
dfic

In [ ]:
dfic['flatDiscreteWorld'][10]
#dfic.loc[:,'w_000':'w_233']
np.array(list(dfic.filter(regex='^w_').iloc[13,:]))

In [ ]:
dfic.groupby(['gameID','targetName'])

In [ ]:
## make matrix of world vectors

# each small multiple is a target structure
# rows and columns are repetitions
# each value is a within participant, within structure, within action index, across repetition distance
# each layer is an action
targets = np.sort(df['targetName'].unique())
ppts = np.sort(df['gameID'].unique())
reps = np.sort(df['repetition'].unique())

n_targets = df['targetName'].nunique()
n_ppts = df['gameID'].nunique()
n_reps = df['repetition'].nunique()
n_grid_squares = 13*18

max_actions = df['numBlocks'].max()

dfic = dfi[['targetName','gameID','blockNum','condition','repetition','flatDiscreteWorld']]
dfic = dfic[dfic.condition=='repeated']

n_rows = n_reps
n_cols = n_reps

all_worlds = np.zeros((n_targets, n_ppts, max_actions, n_reps, n_grid_squares))

for i_t, target in enumerate(targets):
    for i_p, ppt in enumerate(ppts):
        for action in range(0, max_actions):
            for rep in range(0, n_reps):
                world_df = dfic[(dfic.targetName==target) &\
                         (dfic.gameID==ppt) &\
                         (dfic.blockNum==action) &\
                         (dfic.repetition==rep)]['flatDiscreteWorld']
                if (not world_df.empty):
                    all_worlds[i_t,i_p,action,rep,:] = world_df.iloc[0]

In [ ]:
importlib.reload(scoring)

In [ ]:
# calculate all distances

# each small multiple is a target structure
# rows and columns are repetitions
# each value is a within participant, within structure, within action index, across repetition distance
# each layer is an action
targets = np.sort(df['targetName'].unique())
ppts = np.sort(df['gameID'].unique())
reps = np.sort(df['repetition'].unique())

n_targets = df['targetName'].nunique()
n_ppts = df['gameID'].nunique()
n_reps = df['repetition'].nunique()
n_grid_squares = 13*18

max_actions = df['numBlocks'].max()

dfic = dfi[['targetName','gameID','blockNum','condition','repetition','flatDiscreteWorld']]
dfic = dfic[dfic.condition=='repeated']

n_rows = n_reps
n_cols = n_reps

all_dists = np.zeros((n_targets, n_ppts, max_actions, n_rows, n_cols))

df_dists = pd.DataFrame(columns=['target', 'participant', 'block_num','rep_a','rep_b','dist'])

for t in range(0, n_targets):
    for p in range(0, n_ppts):
        for a in range(0, max_actions):
            for rep_a in range (0, n_rows): # row dim
                for rep_b in range (0, n_cols): # col dim
                    world_a = all_worlds[t,p,a,rep_a,:]
                    world_b = all_worlds[t,p,a,rep_b,:]
                    all_dists[t,p,a,rep_a,rep_b] = scoring.get_jaccard(world_a,world_b)
#                     df_dists = df_dists.append({'target': t,
#                                     'participant': p, 
#                                     'block_num': a,
#                                     'rep_a': rep_a,
#                                     'rep_b': rep_b,
#                                     'dist':scoring.get_jaccard(world_a,world_b)}, ignore_index=True)

In [ ]:
v_jaccard = np.vectorize(scoring.get_jaccard)

In [ ]:
img = plt.matshow(all_dists[1,3,8,:,:])
img.set_cmap('hot')
plt.axis('off')
plt.colorbar()

In [ ]:
for action in range(0, max_actions):
    action_dists = all_dists[1,:,action,:,:]
    img.set_cmap('hot')
    norm = plt.Normalize(0, 1)
    img = plt.matshow(np.nanmean(action_dists, axis=0), norm=norm)
    plt.axis('off')
    plt.colorbar()

In [ ]:
plt.set_cmap('hot')
cmap = plt.cm.get_cmap()
cmap.set_bad(color='green')
norm = plt.Normalize(0, 1)

for action in range(0, max_actions):
    img = plt.matshow(all_dists[0,5,action,:,:], cmap = cmap, norm=norm)
    plt.axis('off')
    plt.colorbar()

In [ ]:
#all_dists = (n_targets, n_ppts, max_actions, n_rows, n_cols)

In [ ]:
mean_dists_ppts = np.mean(all_dists[:,:,:,:,:], axis = 1)
mean_dists_ppts.shape

In [ ]:
df_dists['reps'] = df_dists.apply(lambda row: str(int(row.rep_a)) + ' to ' + str(int(row.rep_b)), axis = 1) 

In [ ]:
#fig = plt.figure(figsize=(10,6))
g = sns.FacetGrid(df_dists, row="target", hue="rep_a")
g.map(sns.scatterplot, "block_num", "dist")

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 16)
sns.set(style="ticks", rc={"lines.linewidth": 1})
g = sns.FacetGrid(df_dists, col="target", hue="reps", col_wrap=2, height=8)
g.map(sns.scatterplot, "block_num", "dist", s=15)
g.map(sns.lineplot, "block_num", "dist")


#g.map(sns.lineplot, "block_num", "dist")

In [ ]:
df_dists.reps.unique()

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 16)
sns.set(style="ticks", rc={"lines.linewidth": 1})
g = sns.FacetGrid(df_dists[(df_dists.target==0) & (df_dists.participant.isin(range(0,5)))], col="reps", hue="participant", col_wrap=4, height=6)
g.map(sns.scatterplot, "block_num", "dist", s=15)
g.map(sns.pointplot, "block_num", "dist")


In [ ]:
target = 0

fig = plt.figure(figsize=(10, 20))

for t in range(0, n_targets): 
    plt.subplot(4,2,t+1)
    plt.plot(mean_dists_ppts[t,:,0,1], label="1 to 2", color=line_cmap[0])
    #plt.plot(mean_dists_ppts[t,:,0,2], label="1 to 3", color=line_cmap[1])
    #plt.plot(mean_dists_ppts[t,:,0,3], label="1 to 4", color=line_cmap[2])
    plt.plot(mean_dists_ppts[t,:,1,2], label="2 to 3", color=line_cmap[3])
    #plt.plot(mean_dists_ppts[t,:,1,3], label="2 to 4", color=line_cmap[4])
    plt.plot(mean_dists_ppts[t,:,2,3], label="3 to 4", color=line_cmap[5])
plt.legend()
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

In [ ]:
target = 0

fig = plt.figure(figsize=(10, 20))

for t in range(0, n_targets): 
    plt.subplot(4,2,t+1)
    plt.plot(mean_dists_ppts[t,:,0,1], label="1 to 2", color=line_cmap[0])
    plt.scatter(x = all_dists[:,t,1,0,1], label="1 to 2", color=line_cmap[0])
    #plt.plot(mean_dists_ppts[t,:,0,2], label="1 to 3", color=line_cmap[1])
    #plt.plot(mean_dists_ppts[t,:,0,3], label="1 to 4", color=line_cmap[2])
    plt.plot(mean_dists_ppts[t,:,1,2], label="2 to 3", color=line_cmap[3])
    #plt.plot(mean_dists_ppts[t,:,1,3], label="2 to 4", color=line_cmap[4])
    plt.plot(mean_dists_ppts[t,:,2,3], label="3 to 4", color=line_cmap[5])
plt.legend()
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

At each time point
Relative proportion of full/partial/zero overlap
Each point has a distribution of Jaccard numbers


Over time, integration- area under curves

In [ ]:
target = 0

fig = plt.figure(figsize=(10, 20))

for t in range(0, n_targets): 
    plt.subplot(4,2,t+1)
    plt.plot(all_dists[t,1,:,0,1], label="1 to 2", color=line_cmap[0])
    plt.plot(all_dists[t,1,:,0,2], label="1 to 3", color=line_cmap[1])
    plt.plot(all_dists[t,1,:,0,3], label="1 to 4", color=line_cmap[2])
    plt.plot(all_dists[t,1,:,1,2], label="2 to 3", color=line_cmap[3])
    plt.plot(all_dists[t,1,:,1,3], label="2 to 4", color=line_cmap[4])
    plt.plot(all_dists[t,1,:,2,3], label="3 to 4", color=line_cmap[5])
plt.legend()
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    


## Test Jaccard

In [ ]:
arr1, arr2 = np.zeros((13,18)), np.zeros((13,18))

arr1[0:2,6:10] = 1
plt.matshow(arr1)
print('arr1: ', np.count_nonzero(arr1))

arr2[0:2,7:11] = 1
print('arr2: ', np.count_nonzero(arr1))
plt.matshow(arr2)

print('Jaccard: ', scoring.get_jaccard(arr1,arr2))

In [ ]:
importlib.reload(rda)

all_dists, all_values = rda.rda_from_df(dfi[(dfi.gameID == ppts[0])], 
    row_col='repetition', 
    value_col='flatDiscreteWorld',
    small_multiple_col='targetName', 
    layer_col='blockNum')

In [ ]:
all_dists.shape

In [ ]:
"afa " +\
str("!E") +\
"f"